In [1]:
#libraries 
import pandas as pd
import gzip
from tqdm import tqdm

from datetime import datetime
from datetime import date, time
from dateutil.parser import parse
import numpy as np
import pandas as pd


from bokeh.plotting import figure, output_notebook, show,output_file,gridplot
from bokeh.models import DatetimeTickFormatter,HoverTool,Span

import json
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [3]:
#load the dataframe for a given product only (is it possible?)
pickle_off = open("meta_electronics.pickle","rb")
meta = pickle.load(pickle_off)
meta

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
5,0558835155,http://ecx.images-amazon.com/images/I/21rEirnd...,Use these high quality headphones for internet...,"[[Electronics, Accessories & Supplies, Audio &...",Polaroid Pbm2200 PC / Gaming Stereo Headphones...,13.95,NaN,NaN,NaN
6,0594012015,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[[Electronics, eBook Readers & Accessories, Po...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,32.10,NaN,"{'also_bought': ['B0032UP32K', 'B0034ZRAW4', '...",Barnes &amp; Noble
7,0594017580,http://ecx.images-amazon.com/images/I/41CqeMM0...,"NOOK cover, designed by Jonathon Adler.\nFits ...","[[Electronics, eBook Readers & Accessories, Sk...",Jonathan Adler Punctuation Cover in Black,78.83,NaN,"{'also_viewed': ['161685572X', 'B00G3KA7RW', '...",NaN
8,0594450209,http://ecx.images-amazon.com/images/I/31KkpByX...,Keep your tablet safe with this Barnes &amp; N...,"[[Electronics, eBook Readers & Accessories, Co...",Barnes &amp; Noble 7&quot; Lautner Chocolate N...,15.55,NaN,"{'also_viewed': ['1615527672', '161552763X', '...",NaN
9,0594287995,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,"[[Electronics, eBook Readers & Accessories, Co...",Kate Spade Rain or Shine Magazine Cover for No...,NaN,NaN,NaN,NaN


In [48]:
meta['sub_category_1']=list(map(lambda x:x[0][1],meta.categories))
meta['sub_category_0']=list(map(lambda x:x[0][0],meta.categories))


In [109]:
len(meta.salesRank[26])

0

In [117]:
sales_cat=[]
sales_rank=[]
for i in range(len(meta)):
    if (type(meta.salesRank[i])==dict) :
        if(len(meta.salesRank[i])>0):
            sales_cat.append(list(meta.salesRank[i].keys())[0])
            sales_rank.append(list(meta.salesRank[i].values())[0])
        else:
            sales_cat.append('unkown')
            sales_rank.append(0)
    else:
        sales_cat.append('unkown')
        sales_rank.append(0)

In [126]:
[len(meta),len(sales_rank),len(sales_cat)]

[498196, 498196, 498196]

In [121]:
meta['sales_category']=sales_cat
meta['sales_rank']=sales_rank

In [122]:
meta.head(1)

,asin,imUrl,description,categories,title,price,salesRank,related,brand,sub_category_1,sub_category_0,sales_category,sales_rank
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN,Computers & Accessories,Electronics,unkown,0


In [124]:
#extract only the metadata we need 
meta_use =meta [['asin','title','brand','price','sub_category_0','sub_category_1','sales_category','sales_rank']]

In [125]:
meta_use

,asin,title,brand,price,sub_category_0,sub_category_1,sales_category,sales_rank
0,0132793040,Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,Electronics,Computers & Accessories,unkown,0
1,0321732944,Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,Electronics,Computers & Accessories,unkown,0
2,0439886341,Digital Organizer and Messenger,NaN,8.15,Electronics,Computers & Accessories,Electronics,144944
3,0511189877,CLIKR-5 Time Warner Cable Remote Control UR5U-...,NaN,23.36,Electronics,Accessories & Supplies,unkown,0
4,0528881469,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
5,0558835155,Polaroid Pbm2200 PC / Gaming Stereo Headphones...,NaN,13.95,Electronics,Accessories & Supplies,unkown,0
6,0594012015,Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble,32.10,Electronics,eBook Readers & Accessories,unkown,0
7,0594017580,Jonathan Adler Punctuation Cover in Black,NaN,78.83,Electronics,eBook Readers & Accessories,unkown,0
8,0594450209,Barnes &amp; Noble 7&quot; Lautner Chocolate N...,NaN,15.55,Electronics,eBook Readers & Accessories,unkown,0
9,0594287995,Kate Spade Rain or Shine Magazine Cover for No...,NaN,NaN,Electronics,eBook Readers & Accessories,unkown,0


In [127]:
#load the dataframe for a given product only (is it possible?)
pickle_off = open("Electronics.pickle","rb")
df = pickle.load(pickle_off)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,year,length_review
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,2013-06-02,2013,156
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,2010-11-25,2010,446
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,2010-09-09,2010,888
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,2010-11-24,2010,449
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,2011-09-29,2011,202
5,A2JXAZZI9PHK9Z,0594451647,"Billy G. Noland ""Bill Noland""","[3, 3]",I am using this with a Nook HD+. It works as d...,5.0,HDMI Nook adapter cable,1388707200,2014-01-03,2014,22
6,A2P5U7BDKKT7FW,0594451647,Christian,"[0, 0]",The cable is very wobbly and sometimes disconn...,2.0,Cheap proprietary scam,1398556800,2014-04-27,2014,22
7,AAZ084UMH8VZ2,0594451647,"D. L. Brown ""A Knower Of Good Things""","[0, 0]",This adaptor is real easy to setup and use rig...,5.0,A Perfdect Nook HD+ hook up,1399161600,2014-05-04,2014,120
8,AEZ3CR6BKIROJ,0594451647,Mark Dietter,"[0, 0]",This adapter easily connects my Nook HD 7&#34;...,4.0,A nice easy to use accessory.,1405036800,2014-07-11,2014,88
9,A3BY5KCNQZXV5U,0594451647,Matenai,"[3, 3]",This product really works great but I found th...,5.0,This works great but read the details...,1390176000,2014-01-20,2014,185


In [128]:
#merge 
df_com= pd.merge(df,meta_use,how='left',on='asin')

In [129]:
df_com

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,year,length_review,title,brand,price,sub_category_0,sub_category_1,sales_category,sales_rank
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,2013-06-02,2013,156,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,2010-11-25,2010,446,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,2010-09-09,2010,888,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,2010-11-24,2010,449,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,2011-09-29,2011,202,Rand McNally 528881469 7-inch Intelliroute TND...,NaN,299.99,Electronics,GPS & Navigation,unkown,0
5,A2JXAZZI9PHK9Z,0594451647,"Billy G. Noland ""Bill Noland""","[3, 3]",I am using this with a Nook HD+. It works as d...,5.0,HDMI Nook adapter cable,1388707200,2014-01-03,2014,22,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,NaN,49.95,Electronics,Computers & Accessories,unkown,0
6,A2P5U7BDKKT7FW,0594451647,Christian,"[0, 0]",The cable is very wobbly and sometimes disconn...,2.0,Cheap proprietary scam,1398556800,2014-04-27,2014,22,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,NaN,49.95,Electronics,Computers & Accessories,unkown,0
7,AAZ084UMH8VZ2,0594451647,"D. L. Brown ""A Knower Of Good Things""","[0, 0]",This adaptor is real easy to setup and use rig...,5.0,A Perfdect Nook HD+ hook up,1399161600,2014-05-04,2014,120,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,NaN,49.95,Electronics,Computers & Accessories,unkown,0
8,AEZ3CR6BKIROJ,0594451647,Mark Dietter,"[0, 0]",This adapter easily connects my Nook HD 7&#34;...,4.0,A nice easy to use accessory.,1405036800,2014-07-11,2014,88,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,NaN,49.95,Electronics,Computers & Accessories,unkown,0
9,A3BY5KCNQZXV5U,0594451647,Matenai,"[3, 3]",This product really works great but I found th...,5.0,This works great but read the details...,1390176000,2014-01-20,2014,185,Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,NaN,49.95,Electronics,Computers & Accessories,unkown,0


In [131]:
comp_acc=df_com[df_com.sub_category_1=='Computers & Accessories']

In [132]:
pickling_on = open("Electronics_meta.pickle","wb")
pickle.dump(df_com, pickling_on)
pickling_on.close()